# Apply Vegging to CNN: CIFAR-10 Dataset

### Preparation

In [1]:
# Import modules
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


### Define a CNN Model

In [20]:
# Define a model
class CNN(nn.Module):
    # Initialize the model
    def __init__(self):
        super(CNN, self).__init__()
        
        # Set layers: Function Extraction
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Selt layers:
        self.fc = nn.Linear(32 * 8 * 8,  # Input size  -> Linear transformation to output tensor
                            10)          # Output size  
        
    # Forward
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

### Set Transforms

In [21]:
# Set transforms
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.3, 0.3, 0.3))
])


### Load Datasets

In [22]:
# Load dataset
train_dataset = datasets.CIFAR10(root = './data/0707-CFIAR10', train=True, download=False, transform = train_transform)
test_dataset = datasets.CIFAR10(root = './data/0707-CIFAR10', train=False, download=False, transform=test_transform)

# Set DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### Create an Ensemble Model with CNN Models

In [23]:
# Set number of models
num_models = 5

# Create an ensemble model
models = [CNN().to(device) for _ in range(num_models)]
print(models)

[CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
), CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
), CNN(
  (features): Sequential(
    (0): Conv2d(3, 16, k

### Define Loss Function and Optimizer

In [24]:
# Define Loss Function
criterion = nn.CrossEntropyLoss()

# Define Optimizer
optimizers = [optim.Adam(model.parameters(), lr=0.001) for model in models]

### Fit Models

In [25]:
# Fit a model
num_epochs = 2
for epoch in range(num_epochs):
    # Fit model for training
    for model, optimizer in zip(models, optimizers):
        # Set as a Train mode
        model.train()
        
        # 
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            # Initialize optimizer
            optimizer.zero_grad()
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
    # Evaluate each epoch
    for model in models:
        # Set as an Evaluation mode
        model.eval()
    
    # Initialize lists of prediction and targets
    predictions = []
    targets = []
    
    # Fit model for test
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            # Initialize outputs for the ensemble model
            ensemble_outputs = torch.zeros((images.size(0), 10)).to(device)
            
            for model in models:
                outputs = model(images)
                ensemble_outputs += outputs / num_models  # Accumulate predictions
                
            _, predicted = torch.max(ensemble_outputs.data, 1)
            
            predictions.extend(predicted.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            
    # Calculate accuracy
    accuracy = accuracy_score(targets, predictions)
    print(f'Epoch {epoch+1}/{num_epochs}, Accuracy: {accuracy}')

Epoch 1/2, Accuracy: 0.5493
Epoch 2/2, Accuracy: 0.6059
